In [1]:
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
import numpy as np
import re
import seaborn as sns
import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy.stats import mannwhitneyu
from scipy import stats
import matplotlib.ticker as ticker
import os
from reload_recursive import reload_recursive
import patsy
from pyprocessmacro import Process
from statsmodels.stats.mediation import Mediation
from statsmodels.miscmodels.ordinal_model import OrderedModel
from pingouin import mediation_analysis

from mri_data import file_manager as fm
import helpers

In [2]:
reload_recursive(helpers)

## Setup

In [3]:
drive_root = fm.get_drive_root()
dataroot = drive_root / "3Tpioneer_bids"
data_dir = Path("/home/srs-9/Projects/ms_mri/data")
fig_path = Path("/home/srs-9/Projects/ms_mri/analysis/thalamus/figures_tables/cohort_characteristics")

choroid_volumes = pd.read_csv(
    "/home/srs-9/Projects/ms_mri/data/choroid_aschoplex_volumes.csv", index_col="subid"
).dropna()
tiv = pd.read_csv("/home/srs-9/Projects/ms_mri/data/tiv_data.csv", index_col="subid")

df = pd.read_csv(
    "/home/srs-9/Projects/ms_mri/data/clinical_data_processed.csv", index_col="subid"
)
sdmt = pd.read_csv("/home/srs-9/Projects/ms_mri/analysis/thalamus/SDMT_sheet.csv", index_col="subid")
df = df.join([choroid_volumes, tiv, sdmt['SDMT']])
df['SDMT'] = pd.to_numeric(df['SDMT'], errors='coerce')

df_z = df.copy()
numeric_cols = df.select_dtypes(include='number').columns
df_z[numeric_cols] = df_z[numeric_cols].apply(stats.zscore, nan_policy="omit")


In [4]:
df_thomas = pd.read_csv(data_dir / "hipsthomas_vols.csv", index_col="subid")
cols_orig = df_thomas.columns
new_colnames = {}
for col in df_thomas.columns:
    new_col = re.sub(r"(\d+)-([\w-]+)", r"\2_\1", col)
    new_col = re.sub("-", "_", new_col)
    new_colnames[col] = new_col
df_thomas = df_thomas.rename(columns=new_colnames)
df_thomas_norm = df_thomas.apply(lambda col: col / df_thomas['THALAMUS_1'])
df_thomas_z = df_thomas.apply(stats.zscore, nan_policy="omit")
df_thomas_norm_z = df_thomas_norm.apply(stats.zscore, nan_policy="omit")

df_thomas_left = pd.read_csv(data_dir / "hipsthomas_left_vols.csv", index_col="subid")
df_thomas_left = df_thomas_left.rename(columns=new_colnames)
df_thomas_left_z = df_thomas_left.apply(stats.zscore, nan_policy="omit")

df_thomas_right = pd.read_csv(data_dir / "hipsthomas_right_vols.csv", index_col="subid")
df_thomas_right = df_thomas_right.rename(columns=new_colnames)
df_thomas_right_z = df_thomas_right.apply(stats.zscore, nan_policy="omit")


thalamic_nuclei = [2, 4, 5, 6, 7, 8, 9, 10, 11, 12]
thalamic_nuclei_str = [str(i) for i in thalamic_nuclei]

hips_thomas_ref = pd.read_csv(
    "/home/srs-9/Projects/ms_mri/data/hipsthomas_struct_index.csv", index_col="index"
)['struct']
# hips_thomas_ref.rename(columns={"struct": "struct_name"}, inplace=True)

choroid_dists = pd.read_csv(data_dir / "centroid-choroid_SDT.csv", index_col="subid")
ventricle_dists = pd.read_csv(
    data_dir / "centroid-ventricle_SDT.csv", index_col="subid"
)

mni_choroid_dists = pd.read_csv("/home/srs-9/Projects/ms_mri/data/mni-centroid-choroid_SDT2.csv")

def combine_nuclei(df):
    df2 = pd.DataFrame()
    df2['anterior'] = df['AV_2']
    df2['ventral'] = df['VA_4'] + df['VLa_5'] + df['VLP_6'] + df['VPL_7']
    df2['intralaminar'] = df['CM_11'] 
    df2['medial'] = df['MD_Pf_12']
    df2['posterior'] = df['Pul_8'] + df['LGN_9'] + df['MGN_10']
    df2['THALAMUS_1'] = df['THALAMUS_1']
    return df2

df_thomas2 = combine_nuclei(df_thomas)
df_thomas2_z = df_thomas2.apply(stats.zscore, nan_policy="omit")

df_full = df.join([df_thomas])

## Functions

In [5]:
def mean_sd(df, column, cond=None):
    if cond is None:
        cond = (df.index.isin(df.index))
    
    return df.loc[cond, column].mean(), df.loc[cond, column].std()

def convert(items, factor):
    return [item*factor for item in items]

In [6]:
def percent_cat(df, column, cat, cond=None):
    if cond is None:
        cond = (df.index.isin(df.index))
    
    return sum(cond(df) & (df[column] == cat)) / len(df[cond(df)])

In [7]:
def rms_cond(df):
    return df['dz_type5']=="RMS"

def pms_cond(df):
    return df['dz_type5']=="PMS"

def oind_cond(df):
    return df['dz_type5']=="OIND"

def nind_cond(df):
    return df['dz_type5']=="NIND"

## Descriptive Stats

In [9]:
print(f"RMS: {sum(df["dz_type5"]=="RMS")}")
print(f"PMS: {sum(df["dz_type5"]=="PMS")}")
print(f"NIND: {sum(df["dz_type5"]=="NIND")}")
print(f"OIND: {sum(df["dz_type5"]=="OIND")}")

RMS: 371
PMS: 97
NIND: 49
OIND: 41


### Demographic and Clinical Characteristics

In [10]:
print("Age\n---")
print("RMS:  {:0.2f} ± {:0.2f}".format(*mean_sd(df, "age", cond=rms_cond)))
print("PMS:  {:0.2f} ± {:0.2f}".format(*mean_sd(df, "age", cond=pms_cond)))
print("NIND: {:0.2f} ± {:0.2f}".format(*mean_sd(df, "age", cond=nind_cond)))
print("OIND: {:0.2f} ± {:0.2f}".format(*mean_sd(df, "age", cond=oind_cond)))

print("\n")

print("Sex, Female\n-----------")
print("RMS:  {:0.2f}".format(percent_cat(df, "sex", "Female", cond=rms_cond)))
print("PMS:  {:0.2f}".format(percent_cat(df, "sex", "Female", cond=pms_cond)))
print("NIND: {:0.2f}".format(percent_cat(df, "sex", "Female", cond=nind_cond)))
print("OIND: {:0.2f}".format(percent_cat(df, "sex", "Female", cond=oind_cond)))

print("\n")

print("dzdur\n----")
print("RMS:  {:0.1f} ± {:0.1f}".format(*mean_sd(df, "dzdur", cond=rms_cond)))
print("PMS:  {:0.1f} ± {:0.1f}".format(*mean_sd(df, "dzdur", cond=pms_cond)))
print("NIND: {:0.1f} ± {:0.1f}".format(*mean_sd(df, "dzdur", cond=nind_cond)))
print("OIND: {:0.1f} ± {:0.1f}".format(*mean_sd(df, "dzdur", cond=oind_cond)))

print("\n")

print("EDSS\n----")
print("RMS:  {:0.1f} ± {:0.1f}".format(*mean_sd(df, "EDSS", cond=rms_cond)))
print("PMS:  {:0.1f} ± {:0.1f}".format(*mean_sd(df, "EDSS", cond=pms_cond)))
print("NIND: {:0.1f} ± {:0.1f}".format(*mean_sd(df, "EDSS", cond=nind_cond)))
print("OIND: {:0.1f} ± {:0.1f}".format(*mean_sd(df, "EDSS", cond=oind_cond)))

Age
---
RMS:  45.01 ± 11.77
PMS:  58.26 ± 9.02
NIND: 49.13 ± 11.55
OIND: 49.98 ± 11.94


Sex, Female
-----------
RMS:  0.84
PMS:  0.59
NIND: 0.82
OIND: 0.80


dzdur
----
RMS:  10.4 ± 8.9
PMS:  19.2 ± 11.8
NIND: 9.5 ± 10.7
OIND: 8.2 ± 9.2


EDSS
----
RMS:  2.2 ± 1.5
PMS:  4.9 ± 1.9
NIND: 2.5 ± 1.8
OIND: 2.6 ± 1.7


In [9]:
age = {
    "RMS": "{:0.2f} ± {:0.2f}".format(*mean_sd(df, "age", cond=rms_cond)),
    "PMS": "{:0.2f} ± {:0.2f}".format(*mean_sd(df, "age", cond=pms_cond)),
    "OIND": "{:0.2f} ± {:0.2f}".format(*mean_sd(df, "age", cond=oind_cond)),
    "NIND": "{:0.2f} ± {:0.2f}".format(*mean_sd(df, "age", cond=nind_cond))
}

sex = {
    "RMS": "{:0.2f}".format(percent_cat(df, "sex", "Female", cond=rms_cond)),
    "PMS": "{:0.2f}".format(percent_cat(df, "sex", "Female", cond=pms_cond)),
    "OIND": "{:0.2f}".format(percent_cat(df, "sex", "Female", cond=oind_cond)),
    "NIND": "{:0.2f}".format(percent_cat(df, "sex", "Female", cond=nind_cond))
}

dzdur = {
    "RMS": "{:0.2f} ± {:0.2f}".format(*mean_sd(df, "dzdur", cond=rms_cond)),
    "PMS": "{:0.2f} ± {:0.2f}".format(*mean_sd(df, "dzdur", cond=pms_cond)),
    "OIND": "{:0.2f} ± {:0.2f}".format(*mean_sd(df, "dzdur", cond=oind_cond)),
    "NIND": "{:0.2f} ± {:0.2f}".format(*mean_sd(df, "dzdur", cond=nind_cond))
}

EDSS = {
    "RMS": "{:0.2f} ± {:0.2f}".format(*mean_sd(df, "EDSS", cond=rms_cond)),
    "PMS": "{:0.2f} ± {:0.2f}".format(*mean_sd(df, "EDSS", cond=pms_cond)),
    "OIND": "{:0.2f} ± {:0.2f}".format(*mean_sd(df, "EDSS", cond=oind_cond)),
    "NIND": "{:0.2f} ± {:0.2f}".format(*mean_sd(df, "EDSS", cond=nind_cond))
}

SDMT = {
    "RMS": "{:0.2f} ± {:0.2f}".format(*mean_sd(df, "SDMT", cond=rms_cond)),
    "PMS": "{:0.2f} ± {:0.2f}".format(*mean_sd(df, "SDMT", cond=pms_cond)),
    "OIND": "{:0.2f} ± {:0.2f}".format(*mean_sd(df, "SDMT", cond=oind_cond)),
    "NIND": "{:0.2f} ± {:0.2f}".format(*mean_sd(df, "SDMT", cond=nind_cond))
}
# df_demo = pd.DataFrame({"Age": age, "Sex, Female": sex, "Disease Duration": dzdur, "EDSS": EDSS}, 
#                        index=["RMS"])

df_demo = pd.DataFrame([age, sex, dzdur, EDSS, SDMT], 
                       index=["Age","Sex, Female", "Disease Duration", "EDSS", "SDMT"])
df_demo.to_excel(fig_path / "Demographic_and_clinical.xlsx")

### MRI Data

In [21]:
print("TIV\n----")
print("RMS:  {:0.2f} ± {:0.2f}".format(*convert(mean_sd(df, "tiv", cond=rms_cond), 1/1000)))
print("PMS:  {:0.2f} ± {:0.2f}".format(*convert(mean_sd(df, "tiv", cond=pms_cond), 1/1000)))
print("NIND: {:0.2f} ± {:0.2f}".format(*convert(mean_sd(df, "tiv", cond=nind_cond), 1/1000)))
print("OIND: {:0.2f} ± {:0.2f}".format(*convert(mean_sd(df, "tiv", cond=oind_cond), 1/1000)))

print("\n")

print("Thalamus\n----")
print("RMS:  {:0.2f} ± {:0.2f}".format(*mean_sd(df_full, "THALAMUS_1", cond=rms_cond)))
print("PMS:  {:0.2f} ± {:0.2f}".format(*mean_sd(df_full, "THALAMUS_1", cond=pms_cond)))
print("NIND: {:0.2f} ± {:0.2f}".format(*mean_sd(df_full, "THALAMUS_1", cond=nind_cond)))
print("OIND: {:0.2f} ± {:0.2f}".format(*mean_sd(df_full, "THALAMUS_1", cond=oind_cond)))

print("\n")

print("Cortical Thickness\n----")
print("RMS:  {:0.2f} ± {:0.2f}".format(*mean_sd(df_full, "cortical_thickness", cond=rms_cond)))
print("PMS:  {:0.2f} ± {:0.2f}".format(*mean_sd(df_full, "cortical_thickness", cond=pms_cond)))
print("NIND: {:0.2f} ± {:0.2f}".format(*mean_sd(df_full, "cortical_thickness", cond=nind_cond)))
print("OIND: {:0.2f} ± {:0.2f}".format(*mean_sd(df_full, "cortical_thickness", cond=oind_cond)))

print("\n")

print("lesion_vol_cubic\n----------------")
print("RMS:  {:0.2f} ± {:0.2f}".format(*mean_sd(df, "t2lv", cond=rms_cond)))
print("PMS:  {:0.2f} ± {:0.2f}".format(*mean_sd(df, "t2lv", cond=pms_cond)))
print("NIND: {:0.2f} ± {:0.2f}".format(*mean_sd(df, "t2lv", cond=nind_cond)))
print("OIND: {:0.2f} ± {:0.2f}".format(*mean_sd(df, "t2lv", cond=oind_cond)))

print("\n")

print("Brain\n----------------")
print("RMS:  {:0.2f} ± {:0.2f}".format(*mean_sd(df, "brain", cond=rms_cond)))
print("PMS:  {:0.2f} ± {:0.2f}".format(*mean_sd(df, "brain", cond=pms_cond)))
print("NIND: {:0.2f} ± {:0.2f}".format(*mean_sd(df, "brain", cond=nind_cond)))
print("OIND: {:0.2f} ± {:0.2f}".format(*mean_sd(df, "brain", cond=oind_cond)))

print("\n")

print("choroid plexus\n----------------")
print("RMS:  {:0.2f} ± {:0.2f}".format(*mean_sd(df, "choroid_volume", cond=rms_cond)))
print("PMS:  {:0.2f} ± {:0.2f}".format(*mean_sd(df, "choroid_volume", cond=pms_cond)))
print("NIND: {:0.2f} ± {:0.2f}".format(*mean_sd(df, "choroid_volume", cond=nind_cond)))
print("OIND: {:0.2f} ± {:0.2f}".format(*mean_sd(df, "choroid_volume", cond=oind_cond)))

print("\n")

print("PRL\n----------------")
print("RMS:  {:0.2f} ± {:0.2f}".format(*mean_sd(df, "PRL", cond=rms_cond)))
print("PMS:  {:0.2f} ± {:0.2f}".format(*mean_sd(df, "PRL", cond=pms_cond)))
print("NIND: {:0.2f} ± {:0.2f}".format(*mean_sd(df, "PRL", cond=nind_cond)))
print("OIND: {:0.2f} ± {:0.2f}".format(*mean_sd(df, "PRL", cond=oind_cond)))

TIV
----
RMS:  1490.62 ± 128.50
PMS:  1505.32 ± 127.66
NIND: 1533.21 ± 156.34
OIND: 1504.88 ± 140.66


Thalamus
----
RMS:  9223.33 ± 1499.16
PMS:  7912.53 ± 1758.99
NIND: 10034.03 ± 1224.43
OIND: 9683.56 ± 1334.43


Cortical Thickness
----
RMS:  2.30 ± 0.11
PMS:  2.19 ± 0.13
NIND: 2.32 ± 0.12
OIND: 2.29 ± 0.12


lesion_vol_cubic
----------------
RMS:  1.57 ± 0.93
PMS:  2.02 ± 0.93
NIND: 1.36 ± 0.81
OIND: 1.19 ± 0.52


Brain
----------------
RMS:  1417.99 ± 77.55
PMS:  1364.32 ± 84.14
NIND: 1424.86 ± 65.71
OIND: 1406.58 ± 81.01


choroid plexus
----------------
RMS:  1647.04 ± 505.99
PMS:  1991.07 ± 435.62
NIND: 1616.80 ± 471.60
OIND: 1703.46 ± 498.00


PRL
----------------
RMS:  0.96 ± 1.91
PMS:  0.64 ± 1.23
NIND: 0.02 ± 0.14
OIND: 0.05 ± 0.31


In [28]:
tiv = {
    "RMS": "{:0.2f} ± {:0.2f}".format(*convert(mean_sd(df, "tiv", cond=rms_cond), 1/1000)),
    "PMS": "{:0.2f} ± {:0.2f}".format(*convert(mean_sd(df, "tiv", cond=pms_cond), 1/1000)),
    "OIND": "{:0.2f} ± {:0.2f}".format(*convert(mean_sd(df, "tiv", cond=oind_cond), 1/1000)),
    "NIND": "{:0.2f} ± {:0.2f}".format(*convert(mean_sd(df, "tiv", cond=nind_cond), 1/1000))
}

thalamus = {
    "RMS": "{:0.2f} ± {:0.2f}".format(*convert(mean_sd(df_full, "THALAMUS_1", cond=rms_cond), 1/1000)),
    "PMS": "{:0.2f} ± {:0.2f}".format(*convert(mean_sd(df_full, "THALAMUS_1", cond=pms_cond), 1/1000)),
    "OIND": "{:0.2f} ± {:0.2f}".format(*convert(mean_sd(df_full, "THALAMUS_1", cond=oind_cond), 1/1000)),
    "NIND": "{:0.2f} ± {:0.2f}".format(*convert(mean_sd(df_full, "THALAMUS_1", cond=nind_cond), 1/1000))
}

cortical_thickness = {
    "RMS": "{:0.2f} ± {:0.2f}".format(*mean_sd(df, "cortical_thickness", cond=rms_cond)),
    "PMS": "{:0.2f} ± {:0.2f}".format(*mean_sd(df, "cortical_thickness", cond=pms_cond)),
    "OIND": "{:0.2f} ± {:0.2f}".format(*mean_sd(df, "cortical_thickness", cond=oind_cond)),
    "NIND": "{:0.2f} ± {:0.2f}".format(*mean_sd(df, "cortical_thickness", cond=nind_cond))
}

t2lv = {
    "RMS": "{:0.2f} ± {:0.2f}".format(*mean_sd(df, "t2lv", cond=rms_cond)),
    "PMS": "{:0.2f} ± {:0.2f}".format(*mean_sd(df, "t2lv", cond=pms_cond)),
    "OIND": "{:0.2f} ± {:0.2f}".format(*mean_sd(df, "t2lv", cond=oind_cond)),
    "NIND": "{:0.2f} ± {:0.2f}".format(*mean_sd(df, "t2lv", cond=nind_cond))
}

brain = {
    "RMS": "{:0.2f} ± {:0.2f}".format(*mean_sd(df, "brain", cond=rms_cond)),
    "PMS": "{:0.2f} ± {:0.2f}".format(*mean_sd(df, "brain", cond=pms_cond)),
    "OIND": "{:0.2f} ± {:0.2f}".format(*mean_sd(df, "brain", cond=oind_cond)),
    "NIND": "{:0.2f} ± {:0.2f}".format(*mean_sd(df, "brain", cond=nind_cond))
}

choroid_volume = {
    "RMS": "{:0.2f} ± {:0.2f}".format(*convert(mean_sd(df, "choroid_volume", cond=rms_cond), 1/1000)),
    "PMS": "{:0.2f} ± {:0.2f}".format(*convert(mean_sd(df, "choroid_volume", cond=pms_cond), 1/1000)),
    "OIND": "{:0.2f} ± {:0.2f}".format(*convert(mean_sd(df, "choroid_volume", cond=oind_cond), 1/1000)),
    "NIND": "{:0.2f} ± {:0.2f}".format(*convert(mean_sd(df, "choroid_volume", cond=nind_cond), 1/1000))
}

PRL = {
    "RMS": "{:0.2f} ± {:0.2f}".format(*mean_sd(df, "PRL", cond=rms_cond)),
    "PMS": "{:0.2f} ± {:0.2f}".format(*mean_sd(df, "PRL", cond=pms_cond)),
    "OIND": "{:0.2f} ± {:0.2f}".format(*mean_sd(df, "PRL", cond=oind_cond)),
    "NIND": "{:0.2f} ± {:0.2f}".format(*mean_sd(df, "PRL", cond=nind_cond))
}

df_mri = pd.DataFrame([tiv, brain, thalamus, cortical_thickness, choroid_volume, t2lv, PRL], 
                       index=["ICV", "Brain", "Thalamus", "Cortical Thickness", "CP Volume", "T2LV", "PRL Count"])
df_mri.to_excel(fig_path / "MRI_Features.xlsx")

### HIPS-THOMAS

In [35]:
for nuc in hips_thomas_ref[~hips_thomas_ref.index.isin(thalamic_nuclei)]:
    print(f"{nuc}\n----")
    print("RMS:  {:0.2f} ± {:0.2f}".format(*mean_sd(df_full, nuc, cond=rms_cond)))
    print("PMS:  {:0.2f} ± {:0.2f}".format(*mean_sd(df_full, nuc, cond=pms_cond)))
    print("NIND: {:0.2f} ± {:0.2f}".format(*mean_sd(df_full, nuc, cond=nind_cond)))
    print("OIND: {:0.2f} ± {:0.2f}".format(*mean_sd(df_full, nuc, cond=oind_cond)))
    print("\n")


THALAMUS_1
----
RMS:  9223.33 ± 1499.16
PMS:  7912.53 ± 1758.99
NIND: 10034.03 ± 1224.43
OIND: 9683.56 ± 1334.43


Hb_13
----
RMS:  30.78 ± 7.84
PMS:  27.51 ± 9.29
NIND: 32.11 ± 9.90
OIND: 33.47 ± 5.92


MTT_14
----
RMS:  76.84 ± 13.79
PMS:  75.82 ± 12.99
NIND: 80.76 ± 20.51
OIND: 81.37 ± 13.46


Acc_26
----
RMS:  1166.61 ± 193.26
PMS:  1066.15 ± 222.50
NIND: 1194.83 ± 180.59
OIND: 1209.06 ± 188.27


Cau_27
----
RMS:  6453.81 ± 899.96
PMS:  6048.56 ± 1039.32
NIND: 6660.39 ± 878.88
OIND: 6635.59 ± 777.81


Cla_28
----
RMS:  1337.42 ± 264.75
PMS:  1141.62 ± 291.30
NIND: 1384.52 ± 251.67
OIND: 1371.41 ± 253.14


GPe_29
----
RMS:  669.76 ± 89.41
PMS:  659.26 ± 108.72
NIND: 711.01 ± 91.45
OIND: 696.77 ± 97.61


GPi_30
----
RMS:  365.52 ± 56.20
PMS:  363.99 ± 67.83
NIND: 404.48 ± 56.26
OIND: 381.79 ± 57.24


Put_31
----
RMS:  8943.82 ± 1146.15
PMS:  8544.07 ± 1228.48
NIND: 9495.52 ± 1276.96
OIND: 9069.37 ± 1255.89


RN_32
----
RMS:  482.11 ± 58.53
PMS:  456.74 ± 65.51
NIND: 503.17 ± 72.04
OI

In [36]:
all_nuclei_vols = {}
for nuc in hips_thomas_ref[thalamic_nuclei]:
    all_nuclei_vols[nuc] = {
        "RMS": "{:0.2f} ± {:0.2f}".format(*mean_sd(df_full, nuc, cond=rms_cond)),
        "PMS": "{:0.2f} ± {:0.2f}".format(*mean_sd(df_full, nuc, cond=pms_cond)),
        "OIND": "{:0.2f} ± {:0.2f}".format(*mean_sd(df_full, nuc, cond=oind_cond)),
        "NIND": "{:0.2f} ± {:0.2f}".format(*mean_sd(df_full, nuc, cond=nind_cond))
    }

df_thomas_report = pd.DataFrame(
    [nuclei for nuclei in all_nuclei_vols.values()],
    index=all_nuclei_vols.keys()
)
df_thomas_report.to_excel(fig_path / "thalamic_nuclei_lrmean.xlsx")

In [37]:
all_nuclei_vols = {}
for nuc in hips_thomas_ref[~hips_thomas_ref.index.isin(thalamic_nuclei)]:
    all_nuclei_vols[nuc] = {
        "RMS": "{:0.2f} ± {:0.2f}".format(*mean_sd(df_full, nuc, cond=rms_cond)),
        "PMS": "{:0.2f} ± {:0.2f}".format(*mean_sd(df_full, nuc, cond=pms_cond)),
        "OIND": "{:0.2f} ± {:0.2f}".format(*mean_sd(df_full, nuc, cond=oind_cond)),
        "NIND": "{:0.2f} ± {:0.2f}".format(*mean_sd(df_full, nuc, cond=nind_cond))
    }

df_thomas_report = pd.DataFrame(
    [nuclei for nuclei in all_nuclei_vols.values()],
    index=all_nuclei_vols.keys()
)
df_thomas_report.to_excel(fig_path / "deep_grey_lrmean.xlsx")